In [1]:
# !pip install selenium

In [2]:
url_sector = "https://www.idx.co.id/umbraco/Surface/Helper/GetSectors" 

In [3]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [4]:
service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get(url_sector)
elements = driver.find_elements(By.TAG_NAME, 'body')
for e in elements:
    pass

In [5]:
from ast import literal_eval

sectors = literal_eval(e.text)

In [6]:
def get_url_emitment_by_sector(sector):
    return "https://www.idx.co.id/umbraco/Surface/StockData/GetSecuritiesStock?sector={}&start=0&length=1000".format(sector)

In [7]:
dataset_emitment = []
tracked = []

for sector in sectors:
    url_emitments = get_url_emitment_by_sector(sector)
    driver.get(url_emitments)
    html_elements = driver.find_elements(By.TAG_NAME, 'body')
    emitments = literal_eval(html_elements[0].text)
    emitments['data']
    for emitment in emitments['data']:
        if emitment['Code'] not in tracked :
            dataset_emitment.append({
                'code': emitment['Code'],
                'name': emitment['Name'],
                'listing_date': emitment['ListingDate'],
                'listing_board': emitment['ListingBoard'],
                'shares': emitment['Shares'],
                'sector': sector
            })
            tracked.append(emitment['Code'])
        else:
            index = tracked.index(emitment['Code'])
            dataset_emitment[index]['sector'] = dataset[index]['sector'] + ',' + sector

In [8]:
len(tracked)

810

In [9]:
df = pd.DataFrame(dataset_emitment)
df.head()

,code,name,listing_date,listing_board,shares,sector
0,ADMG,Polychem Indonesia Tbk,1993-10-20T00:00:00,Utama,3889179559,Basic Materials
1,AGII,Aneka Gas Industri Tbk.,2016-09-28T00:00:00,Utama,3066660000,Basic Materials
2,AKPI,Argha Karya Prima Industry Tbk,1992-12-18T00:00:00,Pengembangan,612248000,Basic Materials
3,ALDO,Alkindo Naratama Tbk.,2011-07-12T00:00:00,Pengembangan,1316856070,Basic Materials
4,ALKA,Alakasa Industrindo Tbk,1990-07-12T00:00:00,Pengembangan,507665055,Basic Materials


In [17]:
from bs4 import BeautifulSoup  

dataset_all_fund = {}
for emitment in dataset_emitment:
    ipot_fund = "https://www.indopremier.com/module/saham/include/fundamental.php?code={}&quarter=5".format(emitment['code'])
    driver.get(ipot_fund)
    html_tables = driver.find_elements(By.TAG_NAME, 'table')
    soup = BeautifulSoup(html_tables[0].get_attribute('innerHTML'), 'html.parser')
    data_this_emitment = {}
    for quarter_i, quarter_data in enumerate(soup.find_all("th", attrs={"align": "right"})):
        data_this_quarter = {}
        for measure_i,measure_data in enumerate(soup.find_all('td',attrs={"align": "left", 'colspan':'2'})):
            measure_values = [cleaned for cleaned in soup.find_all('tr') if len(cleaned.find_all('td', attrs={'colspan': 10})) == 0][1:]
            clean_value = [cleaned for cleaned in measure_values[measure_i].text.split('\n') if cleaned != ''][1:]
            data_this_quarter.update({measure_data.text: clean_value[quarter_i]})
        
        data_this_emitment.update({quarter_data.text: data_this_quarter})
    dataset_all_fund.update({emitment['code']: {
        'report': data_this_emitment,
        'name': emitment['name'],
        'listing_date': emitment['listing_date'],
        'listing_board': emitment['listing_board'],
        'shares': emitment['shares'],
        'sector': emitment['sector'],        
    }})        

In [12]:
dataset_all_fund

{'ADMG': {'report': {'Anlz 2022': {'Last Price': '170 ',
    'Share Out': '3.9 B',
    'Market Cap': '661.2 B',
    'Cash': '302.9 B',
    'Total Asset': '2.9 T',
    'S.T.Borrowing': '464.2 B',
    'L.T.Borrowing': '106.0 B',
    'Total Equity': '2.3 T',
    'Revenue': '2.6 T',
    'Gross Profit': '-5.5 B',
    'Operating Profit': '-94.3 B',
    'Net.Profit': '-379.1 B',
    'EBITDA': '-93.9 B',
    'Interest Expense': '3.1 B',
    'Deviden': '0.00 ',
    'EPS': '-97.20 ',
    'PER': '-1.75 x',
    'BVPS': '581.61 ',
    'PBV': '0.29 x',
    'ROA': '-12.95 %',
    'ROE': '-16.76 %',
    'EV/EBITDA': '-9.89 ',
    'Debt/Equity': '0.25 ',
    'Debt/TotalCap': '0.20 ',
    'Debt/EBITDA': '-6.07 ',
    'EBITDA/IntExps': '-30.19 '},
   '[6M] 2022': {'Last Price': '170 ',
    'Share Out': '3.9 B',
    'Market Cap': '661.2 B',
    'Cash': '302.9 B',
    'Total Asset': '2.9 T',
    'S.T.Borrowing': '464.2 B',
    'L.T.Borrowing': '106.0 B',
    'Total Equity': '2.4 T',
    'Revenue': '1.3 T',

In [18]:
dataframe_all_fund = pd.DataFrame(dataset_all_fund)
dataframe_all_fund.head()

,ADMG,AGII,AKPI,ALDO,ALKA,ALMI,ANTM,APLI,ARCI,AVIA,...,SAFE,SAPX,SDMU,SMDR,TAXI,TMAS,TNCA,TRJA,TRUK,WEHA
report,"{'Anlz 2022': {'Last Price': '170 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '2,170 ', 'Share ...","{'Anlz 2022': {'Last Price': '1,645 ', 'Share ...","{'Anlz 2022': {'Last Price': '750 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '290 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '352 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '2,080 ', 'Share ...","{'Anlz 2022': {'Last Price': '195 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '372 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '835 ', 'Share Ou...",...,"{'Anlz 2022': {'Last Price': '195 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '950 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '64 ', 'Share Out...","{'Anlz 2022': {'Last Price': '2,530 ', 'Share ...","{'Anlz 2022': {'Last Price': '50 ', 'Share Out...","{'Anlz 2022': {'Last Price': '2,140 ', 'Share ...","{'Anlz 2022': {'Last Price': '480 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '280 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '124 ', 'Share Ou...","{'Anlz 2022': {'Last Price': '117 ', 'Share Ou..."
name,Polychem Indonesia Tbk,Aneka Gas Industri Tbk.,Argha Karya Prima Industry Tbk,Alkindo Naratama Tbk.,Alakasa Industrindo Tbk,Alumindo Light Metal Industry,Aneka Tambang Tbk.,Asiaplast Industries Tbk.,Archi Indonesia Tbk.,Avia Avian Tbk.,...,Steady Safe Tbk,Satria Antaran Prima Tbk.,Sidomulyo Selaras Tbk.,Samudera Indonesia Tbk.,Express Transindo Utama Tbk.,Temas Tbk.,Trimuda Nuansa Citra Tbk.,Transkon Jaya Tbk.,Guna Timur Raya Tbk.,WEHA Transportasi Indonesia Tb
listing_date,1993-10-20T00:00:00,2016-09-28T00:00:00,1992-12-18T00:00:00,2011-07-12T00:00:00,1990-07-12T00:00:00,1997-01-02T00:00:00,1997-11-27T00:00:00,2000-05-01T00:00:00,2021-06-28T00:00:00,2021-12-08T00:00:00,...,1994-08-15T00:00:00,2018-10-03T00:00:00,2011-07-12T00:00:00,1999-07-05T00:00:00,2012-11-02T00:00:00,2003-09-07T00:00:00,2018-06-28T00:00:00,2020-08-27T00:00:00,2018-05-23T00:00:00,2007-05-31T00:00:00
listing_board,Utama,Utama,Pengembangan,Pengembangan,Pengembangan,Pengembangan,Utama,Utama,Utama,Utama,...,PENGEMBANGAN,Pengembangan,Pengembangan,Utama,UTAMA,UTAMA,Pengembangan,Utama,PENGEMBANGAN,Utama
shares,3889179559,3066660000,612248000,1316856070,507665055,3816000000,24030764725,1362671400,24835000000,61953555600,...,615145012,833333300,1135225000,3275120000,10223647156,5705150000,421640000,1510200000,435000000,1460554819


In [19]:
import datetime
date = datetime.datetime.now()

str_date = (datetime.datetime.now().strftime("%Y-%m-%d"))

In [20]:
dataframe_all_fund.transpose().to_csv('{}.csv'.format(str_date))